In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "/Users/wolheelee/python/kaggle/PortoSeguroSafeDriverPrediction"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

20171117.ipynb
20171120
20171121
20171123
20171124
20171124_LB287_Mix.ipynb
20171124_strat20_LGBM_999.ipynb
20171125_LB287_kernelfromkaggle.ipynb
20171125_kagglemix_rev3.csv
20171125_work.ipynb
279
280
283
283_XGB
284
DL_TF_NN2.ipynb
LB287_Mix.ipynb
StackingLB285.ipynb
XGBoostCV_LB284.ipynb
catboost_feature_importance.png
input
learn_error.tsv
meta.tsv
old
plotly
test
test_error.tsv
time_left.tsv
train
xgb_submit.csv
xgb_valid.csv
提出csv
ランキング



In [2]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier

from rgf import *     # https://github.com/fukatani/rgf_python

train = pd.read_csv('/Users/wolheelee/python/kaggle/PortoSeguroSafeDriverPrediction/input/train.csv')
test = pd.read_csv('/Users/wolheelee/python/kaggle/PortoSeguroSafeDriverPrediction/input/test.csv')


In [3]:
# The lgbm part of the code is infuenced by https://www.kaggle.com/yekenot/simple-stacker-lb-0-284
# with a little modification
# The output of this model is available under Input datasets section
# Preprocessing 
id_test = test['id'].values
target_train = train['target'].values

train = train.drop(['target','id'], axis = 1)
test = test.drop(['id'], axis = 1)

col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(col_to_drop, axis=1)  
test = test.drop(col_to_drop, axis=1)  

train = train.replace(-1, np.nan)
test = test.replace(-1, np.nan)

cat_features = [a for a in train.columns if a.endswith('cat')]

for column in cat_features:
	temp = pd.get_dummies(pd.Series(train[column]))
	train = pd.concat([train,temp],axis=1)
	train = train.drop([column],axis=1)
    
for column in cat_features:
	temp = pd.get_dummies(pd.Series(test[column]))
	test = pd.concat([test,temp],axis=1)
	test = test.drop([column],axis=1)

print(train.values.shape, test.values.shape)


(595212, 198) (892816, 198)


In [4]:
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=2016).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
#                y_holdout = y[test_idx]

                print ("Fit %s fold %d" % (str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
#                cross_score = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')
#                print("    cross_score: %.5f" % (cross_score.mean()))
                y_pred = clf.predict_proba(X_holdout)[:,1]                

                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict_proba(T)[:,1]
            S_test[:, i] = S_test_i.mean(axis=1)

        results = cross_val_score(self.stacker, S_train, y, cv=3, scoring='roc_auc')
        print("Stacker score: %.5f" % (results.mean()))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict_proba(S_test)[:,1]
        return res

In [5]:
# LightGBM params
lgb_params = {}
lgb_params['learning_rate'] = 0.02
lgb_params['n_estimators'] = 650
lgb_params['max_bin'] = 10
lgb_params['subsample'] = 0.8
lgb_params['subsample_freq'] = 10
lgb_params['colsample_bytree'] = 0.8   
lgb_params['min_child_samples'] = 500
lgb_params['feature_fraction'] = 0.9
lgb_params['num_iterations']=900
lgb_params['bagging_freq'] = 1
lgb_params['seed'] = 200

lgb_params2 = {}
lgb_params2['n_estimators'] = 1090
lgb_params2['learning_rate'] = 0.02
lgb_params2['colsample_bytree'] = 0.3   
lgb_params2['subsample'] = 0.7
lgb_params2['subsample_freq'] = 2
lgb_params2['num_leaves'] = 16
lgb_params2['num_iterations']=900
lgb_params2['feature_fraction'] = 0.9
lgb_params2['bagging_freq'] = 1
lgb_params2['seed'] = 200


lgb_params3 = {}
lgb_params3['n_estimators'] = 1100
lgb_params3['max_depth'] = 4
lgb_params3['num_iterations']=900
lgb_params3['learning_rate'] = 0.02
lgb_params3['feature_fraction'] = 0.9
lgb_params3['bagging_freq'] = 1
lgb_params3['seed'] = 200

In [6]:
lgb_model = LGBMClassifier(**lgb_params)

lgb_model2 = LGBMClassifier(**lgb_params2)

lgb_model3 = LGBMClassifier(**lgb_params3)

In [7]:
log_model = LogisticRegression()
       
stack = Ensemble(n_splits=6,
        stacker = log_model,
        base_models = (lgb_model, lgb_model2, lgb_model3))        
        
y_pred = stack.fit_predict(train, target_train, test)        

sub_1 = pd.DataFrame()
sub_1['id'] = id_test
sub_1['target'] = y_pred

/Users/wolheelee/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/wolheelee/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/wolheelee/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/lightgbm/engine.py:98: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Fit LGBMClassifier fold 1


/Users/wolheelee/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/wolheelee/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/wolheelee/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/lightgbm/engine.py:98: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Fit LGBMClassifier fold 2


/Users/wolheelee/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/wolheelee/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/wolheelee/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/lightgbm/engine.py:98: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


...

Fit LGBMClassifier fold 4


/Users/wolheelee/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/wolheelee/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/wolheelee/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/lightgbm/engine.py:98: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Fit LGBMClassifier fold 5


/Users/wolheelee/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/wolheelee/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/lightgbm/sklearn.py:282: LGBMDeprecationWarning: The `seed` parameter is deprecated and will be removed in next version. Please use `random_state` instead.
  'Please use `random_state` instead.', LGBMDeprecationWarning)
/Users/wolheelee/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/lightgbm/engine.py:98: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Fit LGBMClassifier fold 6
Stacker score: 0.64457


In [11]:
# All these datasets are from different kaggle kernels
stacked_1 = pd.read_csv('/Users/wolheelee/python/kaggle/PortoSeguroSafeDriverPrediction/input/kaggleportosegurosubmissions/stacked_1.csv')
xgb_submit = pd.read_csv('/Users/wolheelee/python/kaggle/PortoSeguroSafeDriverPrediction/input/kaggleportosegurosubmissions/xgb_submit.csv')
Froza_and_Pascal = pd.read_csv('/Users/wolheelee/python/kaggle/PortoSeguroSafeDriverPrediction/input/kaggleportosegurosubmissions/Froza_and_Pascal.csv')
rgf_submit = pd.read_csv('/Users/wolheelee/python/kaggle/PortoSeguroSafeDriverPrediction/input/kaggleportosegurosubmissions/rgf_submit.csv')
gpari = pd.read_csv('/Users/wolheelee/python/kaggle/PortoSeguroSafeDriverPrediction/input/kaggleportosegurosubmissions/gpari.csv')

# Ensemble and create submission 

sub = pd.DataFrame()
sub['id'] = stacked_1['id']
sub['target'] = np.exp(np.mean(
	[	
    sub_1['target'].apply(lambda x: np.log(x)),\
    Froza_and_Pascal['target'].apply(lambda x: np.log(x)),\
    rgf_submit['target'].apply(lambda x: np.log(x)),\
    gpari['target'].apply(lambda x: np.log(x)),\
	stacked_1['target'].apply(lambda x: np.log(x)),\
	xgb_submit['target'].apply(lambda x: np.log(x))\
	], axis =0))

sub.to_csv('sub.csv', index = False, compression = 'gzip')

In [12]:
sub

id    target
0             0  0.045288
1             1  0.043682
2             2  0.040265
3             3  0.029925
4             4  0.057016
5             5  0.069973
6             6  0.032872
7             8  0.061785
8            10  0.076607
9            11  0.088624
10           12  0.049617
11           14  0.040532
12           15  0.066310
13           18  0.068881
14           21  0.071989
15           23  0.041352
16           24  0.042247
17           25  0.083272
18           27  0.030328
19           29  0.086970
20           30  0.054989
21           31  0.081201
22           32  0.086928
23           33  0.029627
24           37  0.044303
25           38  0.049450
26           39  0.123759
27           40  0.064142
28           41  0.043500
29           42  0.033067
...         ...       ...
892786  1487982  0.031993
892787  1487984  0.059759
892788  1487985  0.056577
892789  1487986  0.058823
892790  1487987  0.054244
892791  1487989  0.044521
892792  1487991  0.048435
892793  1487993  0.063935
892794  1487995  0.041439
892795  1487997  0.057869
892796  1487998  0.090635
892797  1487999  0.096722
892798  1488000  0.060491
892799  1488002  0.122240
892800  1488003  0.042420
892801  1488004  0.051115
892802  1488006  0.045888
892803  1488007  0.055408
892804  1488010  0.068727
892805  1488012  0.058772
892806  1488014  0.044804
892807  1488015  0.043060
892808  1488018  0.042639
892809  1488019  0.031890
892810  1488020  0.037956
892811  1488022  0.126593
892812  1488023  0.067648
892813  1488024  0.059381
892814  1488025  0.040244
892815  1488026  0.052784

[892816 rows x 2 columns]